In [5]:
from nilearn import image
from nilearn import glm
from nilearn import plotting
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import os
from joblib import Parallel, delayed

In [6]:
time_repetition = 2

In [7]:
sublist = [
    "06","07","08","09","10","11",
    "13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33",
    "36","37","38","39",
    "41","42","43","44"
    ]

runlist = [
    "01","02","03","04","05"
    ]

In [8]:
all_masks, files = [], []
msk_thrs = 1
for subid in sublist:
    funcdir = os.path.join('/research/Re/StatisticalPainLearning/derivatives/preprocessed', f'sub-{subid}', 'func')
    all_masks += [image.load_img(os.path.join(funcdir, f)).get_fdata() for f in os.listdir(funcdir) if 'MNI152NLin2009cAsym_desc-brain_mask.nii.gz' in f]
    files += [os.path.join(funcdir, f) for f in os.listdir(funcdir) if 'MNI152NLin2009cAsym_desc-brain_mask.nii.gz' in f]
all_masks = np.stack(all_masks)
group_mask = np.where(np.sum(all_masks, axis=0)/all_masks.shape[0] >= msk_thrs, 1, 0)
group_mask = image.new_img_like(files[0], group_mask)
group_mask.to_filename('/research/Re/StatisticalPainLearning/derivatives/group_mask.nii.gz')

/tmp/ipykernel_27393/301209784.py:9: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  group_mask = image.new_img_like(files[0], group_mask)


In [9]:
def run_firstlevel(subid, runid):
    templates = dict(
        func = os.path.join("derivatives","preprocessed","sub-{subid}","func","sub-{subid}_task-tsl_run-{runid}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"),
        confound = os.path.join("derivatives","preprocessed","sub-{subid}","func","sub-{subid}_task-tsl_run-{runid}_desc-confounds_timeseries.tsv"),
        event = os.path.join("BIDSdata","sub-{subid}","func","sub-{subid}_task-tsl_run-{runid}_events.tsv"))
    
    print(f"Processing {templates['func'].format(subid=subid,runid=runid)}")
    
    if not os.path.exists(os.path.join("derivatives","firstlevel",f"sub-{subid}")):
        os.makedirs(os.path.join("derivatives","firstlevel",f"sub-{subid}"))
    
    func = templates["func"].format(subid=subid,runid=runid)
    confound = templates["confound"].format(subid=subid,runid=runid)
    event = templates["event"].format(subid=subid,runid=runid)

    confound_data = pd.read_csv(confound, sep="\t")
    event_data = pd.read_csv(event, sep="\t")
    func_run = image.load_img(func)
    func_run = func_run.slicer[..., 4:]
    n_scans = func_run.shape[-1]

    confound_names = ['csf','white_matter','global_signal',
                    'trans_x', 'trans_y', 'trans_z', 'rot_x','rot_y','rot_z']
    
    regressors = confound_data[confound_names].loc[4:].fillna(0)
    
    design_matrix = glm.first_level.make_first_level_design_matrix(
        frame_times = np.arange(n_scans) * time_repetition,
        events = event_data,
        drift_model = "polynomial",
        add_regs = regressors,
        add_reg_names= confound_names)
    
    plotting.plot_design_matrix(
        design_matrix=design_matrix,
        output_file=os.path.join("derivatives","firstlevel",f"sub-{subid}",f"sub-{subid}_run-{runid}_design_matrix.png"))
    
    first_GLM = glm.first_level.FirstLevelModel(
        t_r = time_repetition,
        noise_model = "ar1",
        standardize = False,
        hrf_model = "glover",
        high_pass = 1/128, 
        minimize_memory = True,
        smoothing_fwhm=8,
        n_jobs=-1)
    
    first_fit = first_GLM.fit(
        run_imgs=func_run,
        design_matrices=design_matrix)

    c_map = first_fit.compute_contrast(
        contrast_def="high-low",
        stat_type='t', 
        output_type='z_score')

    c_map.to_filename(
        os.path.join("derivatives","firstlevel",f"sub-{subid}",f"sub-{subid}_run-{runid}_contrast.nii.gz")
    )

    report = first_fit.generate_report(
        contrasts=["high-low"])
    report.save_as_html(
        os.path.join("derivatives","firstlevel",f"sub-{subid}",f"sub-{subid}_run-{runid}_report.html"))
    return c_map

In [10]:
sj_ls = [f.split('-')[1] for f in os.listdir("/research/Re/StatisticalPainLearning/derivatives/preprocessed") if f.startswith('sub') and not f.endswith('.html')]
sj_to_fl = {sj: [] for sj in sj_ls}
for j in sj_ls:
    fl = [f[20:22] for f in os.listdir(os.path.join("/research/Re/StatisticalPainLearning/derivatives/preprocessed", f"sub-{j}", "func")) if f.endswith("bold.nii.gz")]
    sj_to_fl[j] = fl

In [ ]:
c_map_list = Parallel(n_jobs=8)(
    delayed(run_firstlevel)(subid, runid) for subid in sj_ls for runid in sj_to_fl[subid]
)

Processing derivatives/preprocessed/sub-32/func/sub-32_task-tsl_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Processing derivatives/preprocessed/sub-32/func/sub-32_task-tsl_run-03_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Processing derivatives/preprocessed/sub-32/func/sub-32_task-tsl_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Processing derivatives/preprocessed/sub-32/func/sub-32_task-tsl_run-04_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Processing derivatives/preprocessed/sub-32/func/sub-32_task-tsl_run-05_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Processing derivatives/preprocessed/sub-27/func/sub-27_task-tsl_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Processing derivatives/preprocessed/sub-27/func/sub-27_task-tsl_run-05_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
Processing derivatives/preprocessed/sub-27/func/sub-27_task-tsl_run-04_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz


In [ ]:
design_matrix = pd.DataFrame(
    np.ones(len(c_map_list)),
    columns=["intercept"])

secondlevelmodel = glm.second_level.SecondLevelModel(
    mask_img=group_mask,
    n_jobs=-1)

second_level_model = secondlevelmodel.fit(
    second_level_input=c_map_list,
    design_matrix=design_matrix)


z_map = second_level_model.compute_contrast(
    second_level_contrast="intercept",
    output_type="z_score")

z_map.to_filename(
    os.path.join("/research/Re/StatisticalPainLearning/derivatives/secondlevel/z_map.nii.gz"))

second_level_report = second_level_model.generate_report(
    contrasts=["intercept"],
    title="Results of the second-level analysis",
    alpha = 0.001, 
    cluster_threshold = 10,
    height_control = "fpr",
    display_mode="ortho")
second_level_report.save_as_html("/research/Re/StatisticalPainLearning/derivatives/secondlevel/nilearn_report.html")